In [ ]:
from datasets import load_dataset
import datasets
ds = load_dataset("openai/gsm8k", "main")
train: datasets.Dataset = ds["train"]
prompt_templ = """A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.
User: {0}
Assistant: <think>"""
print(prompt_templ)

In [ ]:
from cs336_alignment.math_baseline import extract_answer
prompts = []
ground_truths = []
for t, data in enumerate(train):
    question = data["question"]
    answer_text = data["answer"]
    answer = extract_answer(answer_text)
    assert answer is not None, f"Could not extract answer from: {answer_text}"
    full_prompt = prompt_templ.format(question)
    prompts.append(full_prompt)
    ground_truths.append(answer)

In [ ]:
import random
index = random.randint(0, len(prompts) - 1)
print("Question:")
print(train[index]["question"])
print("-" * 20)
print("Ground truth answer:")
print(ground_truths[index])
print("-" * 20)
print("Answer Text:")
print(train[index]["answer"])

In [11]:
import pickle
with open("../data/math_baseline_eval_results.pkl", "rb") as f:
    results = pickle.load(f)

In [ ]:
import random
prompts = results["prompts"]
responses = results["responses"]
rewards = results["rewards"]
ground_truths = results["ground_truths"]

index = random.randint(0, len(prompts) - 1)
r = rewards[index]
format_rewards = r["format_reward"]
if r["reward"] <= 0 and format_rewards <= 0:
    print("Prompt:")
    print(prompts[index])
    print("-" * 20)
    print("Response:")
    print(responses[index])
    print("-" * 20)
    print("Ground Truth:")
    print(ground_truths[index])
    print("-" * 20)
    print("Reward:")
    print(rewards[index])

"""Format Reward looks correct, only one issue is that therre are some cases 
where <think/> something <answer> answer </answer> is not satisfied. 
We can improve the format reward by checking for this condition."""

In [ ]:
correct_count = 0
format_correct_count = 0
wrong_count = 0
for t, r in enumerate(rewards):
    formatted_reward = r["format_reward"]
    answer_reward = r["answer_reward"]
    r = r["reward"]
    if formatted_reward > 0 and answer_reward > 0:
        correct_count += 1
    elif formatted_reward > 0 and answer_reward <= 0:
        format_correct_count += 1
    else:
        wrong_count += 1
print(f"Total samples: {len(rewards)}")
print(f"Correct samples: {correct_count}, percent: {correct_count / len(rewards):.4f}")
print(f"Format correct samples: {format_correct_count}, percent: {format_correct_count / len(rewards):.4f}")
print(f"Wrong samples: {wrong_count}, percent: {wrong_count / len(rewards):.4f}")

In [2]:
import random
prompts = results["prompts"]
responses = results["responses"]
rewards = results["rewards"]
ground_truths = results["ground_truths"]
all_indexes = []
for t, r in enumerate(rewards):
    formatted_reward = r["format_reward"]
    answer_reward = r["answer_reward"]
    r = r["reward"]
    if formatted_reward > 0 and answer_reward <= 0:
        all_indexes.append(t)

index = random.choice(all_indexes)
r = rewards[index]
format_rewards = r["format_reward"]
print("Prompt:")
print(prompts[index])
print("-" * 20)
print("Response:")
print(responses[index])
print("-" * 20)
print("Ground Truth:")
print(ground_truths[index])
print("-" * 20)
print("Reward:")
print(rewards[index])

NameError: name 'results' is not defined

In [1]:
response = """First, we need to find the number of male members in the club. Since there are two times as many female members as male members, we can set up the following equation: Number of female members = 2 * Number of male members. We also know that the total number of members in the club is 18. Therefore, Number of male members + Number of female members = 18. Substituting the first equation into the second equation, we get Number of male members + 2 * Number of male members = 18. Solving for Number of male members, we get Number of male members = 6. Therefore, Number of female members = 2 * 6 = 12.</think> <answer> There are 12 female members in the glee club.</answer>
"""
gt = "12"
from cs336_alignment.drgrpo_grader import r1_zero_reward_fn
from math_verify import parse, verify
reward = r1_zero_reward_fn(response, gt, False)
resp_answer = parse(response)
is_correct = verify(resp_answer, gt)
print(f"resp_answer: {resp_answer}, ground truth: {gt}, is_correct: {is_correct}")
print(f"Reward: {reward}")

/home/iceberg/repos/assignment5-alignment/cs336_alignment/drgrpo_grader.py:45: SyntaxWarning: invalid escape sequence '\{'
  m = re.search("^\\\\text\{(?P<text>.+?)\}$", answer)
/home/iceberg/repos/assignment5-alignment/cs336_alignment/drgrpo_grader.py:320: SyntaxWarning: invalid escape sequence '\%'
  string = string.replace("\%", "")
/home/iceberg/repos/assignment5-alignment/cs336_alignment/drgrpo_grader.py:673: SyntaxWarning: invalid escape sequence '\^'
  BAD_REGEXES = ["\^[0-9]+\^", "\^[0-9][0-9]+"]
/home/iceberg/repos/assignment5-alignment/cs336_alignment/drgrpo_grader.py:673: SyntaxWarning: invalid escape sequence '\^'
  BAD_REGEXES = ["\^[0-9]+\^", "\^[0-9][0-9]+"]
/home/iceberg/repos/assignment5-alignment/cs336_alignment/drgrpo_grader.py:753: SyntaxWarning: invalid escape sequence '\d'
  p1 = re.compile("(\d)(,)(\d\d\d)($|\D)")
/home/iceberg/repos/assignment5-alignment/cs336_alignment/drgrpo_grader.py:768: SyntaxWarning: invalid escape sequence '\{'
  m = re.search("^\\\\text\

resp_answer: [12, '12'], ground truth: 12, is_correct: True
Reward: {'format_reward': 1.0, 'answer_reward': 1.0, 'reward': 1.0}
